In [1]:
import numpy as np
import dolfin as dl
from nalger_helper_functions import pointwise_observation_matrix, points_inside_mesh, make_grid_transfer_function

In [2]:
mesh = dl.UnitSquareMesh(11,13)
V = dl.FunctionSpace(mesh, 'CG', 2)
u = dl.Function(V)
u.vector()[:] = np.random.randn(V.dim())

N = 100
d = mesh.geometric_dimension()
pp = np.random.randn(N, d)

inside_mesh_mask = points_inside_mesh(pp, mesh) # <-- True where point is in mesh, false where point is outside

In [3]:
B_in = pointwise_observation_matrix(pp[inside_mesh_mask, :], V) # <-- makes scipy CSR matrix

x1 = np.zeros(N)
for k in range(N):
    if np.all(np.zeros(2) <= pp[k,:]) and np.all(pp[k,:] <= np.ones(2)):
        x1[k] = u(dl.Point(pp[k,:]))

x2 = np.zeros(N)
x2[inside_mesh_mask] = B_in * u.vector()[:]

err_pointwise_observation_matrix = np.linalg.norm(x2 - x1)
print('err_pointwise_observation_matrix=', err_pointwise_observation_matrix)

err_pointwise_observation_matrix= 2.9893669801409083e-16


In [4]:
exterior_fill_value = 3.5

grid_transfer_function = make_grid_transfer_function(pp, V, exterior_fill_value=exterior_fill_value)
x3 = np.zeros(N)
for k in range(N):
    if np.all(np.zeros(2) <= pp[k,:]) and np.all(pp[k,:] <= np.ones(2)):
        x3[k] = u(dl.Point(pp[k,:]))
    else:
        x3[k] = 3.5

x4 = grid_transfer_function(u.vector()[:])

err_grid_transfer_function = np.linalg.norm(x4 - x3)
print('err_grid_transfer_function=', err_grid_transfer_function)

err_grid_transfer_function= 2.9893669801409083e-16
